In [4]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/prompt-to-prompt-main')

Mounted at /content/drive


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/prompt-to-prompt-main/')  # Adjust path as needed
import ptp_utils


In [8]:
!pip install -r /content/drive/MyDrive/prompt-to-prompt-main/requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.31.0
    Uninstalling diffusers-0.31.0:
      Successfully uninstalled diffusers-0.31.0


In [9]:
import torch
from diffusers import DiffusionPipeline
import ptp_utils
from ptp_utils import LocalBlend, AttentionReplace, AttentionRefine, AttentionReweight, get_equalizer
from PIL import Image
import numpy as np

# Load Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_id = "CompVis/ldm-text2im-large-256"
ldm = DiffusionPipeline.from_pretrained(model_id).to(device)

NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 5.0

# Function to display images
def display_images(images):
    for image in images:
        display(Image.fromarray(image))

# Function to run diffusion and display
def run_diffusion_and_display(prompts, controller, latent=None):
    images, latent = ptp_utils.text2image_ldm(ldm, prompts, controller, latent=latent,
                                              num_inference_steps=NUM_DIFFUSION_STEPS,
                                              guidance_scale=GUIDANCE_SCALE)
    display_images(images)
    return images, latent

# Hands-On Task 1: Add a New Object
# Example: Add a tree to a serene beach
prompts = [
    "A serene beach",
    "A serene beach with a tree"
]
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=0.8, self_replace_steps=0.2)
images, latent = run_diffusion_and_display(prompts, controller)

# Hands-On Task 2: Experiment Design
# Customizing product images
prompts = [
    "A red smartphone",
    "A blue smartphone with a logo"
]
controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=0.8, self_replace_steps=0.2)
images, latent = run_diffusion_and_display(prompts, controller)

# Hands-On Task 3: Custom Prompts and Edits
# Example: Modify scenes with specific conditions
base_prompt = "A futuristic car on a highway"
edited_prompts = [
    "A futuristic car on a highway during sunset",
    "A futuristic car on a rainy highway",
    "A futuristic car on a snowy highway"
]
controller = AttentionReplace(edited_prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=0.8, self_replace_steps=0.2)
images, latent = run_diffusion_and_display(edited_prompts, controller)

# Optional: Attention Reweighting for Emphasis
# Adding attention to specific tokens (e.g., "birthday" in "birthday cake")
prompts = ["cake", "birthday cake"]
equalizer = get_equalizer(prompts[1], word_select="birthday", values=(5,))
lb = LocalBlend(prompts, ("cake", ("birthday", "cake")))
controller = AttentionReweight(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=0.8, self_replace_steps=0.4,
                               equalizer=equalizer, local_blend=lb)
images, latent = run_diffusion_and_display(prompts, controller)


ImportError: cannot import name 'LocalBlend' from 'ptp_utils' (/content/drive/MyDrive/prompt-to-prompt-main/ptp_utils.py)